<a href="https://colab.research.google.com/github/LifeHackInnovationsLLC/whisper-video-transcription/blob/main/LHI_WhisperVideoDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# LHI_WhisperVideoDrive.py

In [33]:
# ---
# jupyter:
#   jupytext:
#     formats: ipynb,py:percent
#     text_representation:
#       extension: .py
#       format_name: percent
#       format_version: '1.3'
#       jupytext_version: 1.16.5
#   kernelspec:
#     display_name: Python 3
#     name: python3
# ---

<a href="https://colab.research.google.com/github/LifeHackInnovationsLLC/whisper-video-transcription/blob/main/LHI_WhisperVideoDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Jupytext Initialization (Sync Logic)
Ensure Jupytext is installed and the notebook is paired with the `.py` file.

import subprocess
import sys

def ensure_module(module_name, install_name=None):
    """Install a module if it's not already installed."""
    try:
        __import__(module_name)
        print(f"Module '{module_name}' is already installed.")
    except ImportError:
        install_name = install_name or module_name
        print(f"Module '{module_name}' not found. Installing...")
        subprocess.run([sys.executable, "-m", "pip", "install", install_name], check=True)

Ensure Jupytext is installed
ensure_module("jupytext")

Sync the notebook with its paired `.py` file
try:
    subprocess.run(["jupytext", "--sync", "LHI_WhisperVideoDrive.ipynb"], check=True)
    print("Jupytext synchronization successful.")
except subprocess.CalledProcessError as e:
    print(f"Error during Jupytext synchronization: {e}")

In [34]:
# Handle missing modules and Google Colab environment checks

import subprocess
import sys


# Install and import required modules
required_modules = {
    "google.colab": "google-colab",
    "whisper": "openai-whisper",
    "librosa": "librosa",
    "soundfile": "soundfile"
}

for module, install_name in required_modules.items():
    try:
        __import__(module)
        print(f"Module '{module}' is already installed.")
    except ImportError:
        print(f"Module '{module}' not found. Installing...")
        subprocess.run([sys.executable, "-m", "pip", "install", install_name], check=True)

# Conditional import for Google Colab
try:
    from google.colab import drive
    print("Google Colab environment detected.")
except ImportError:
    print("Google Colab environment not detected. Skipping Colab imports.")

# Import other required modules
import whisper
import librosa
import soundfile as sf



Module 'google.colab' is already installed.
Module 'whisper' is already installed.
Module 'librosa' is already installed.
Module 'soundfile' is already installed.
Google Colab environment detected.



#📼 OpenAI Whisper + Google Drive Video Transcription

📺 Getting started video: https://youtu.be/YGpYinji7II

###This application will extract audio from all the video files in a Google Drive folder and create a high-quality transcription with OpenAI's Whisper automatic speech recognition system.

*Note: This requires giving the application permission to connect to your drive. Only you will have access to the contents of your drive, but please read the warnings carefully.*

This notebook application:
1. Connects to your Google Drive when you give it permission.
2. Creates a WhisperVideo folder and three subfolders (ProcessedVideo, AudioFiles and TextFiles.)
3. When you run the application it will search for all the video files (.mp4, .mov, mkv and .avi) in your WhisperVideo folder, transcribe them and then move the file to WhisperVideo/ProcessedVideo and save the transcripts to WhisperVideo/TextFiles. It will also add a copy of the new audio file to WhisperVideo/AudioFiles

###**For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Note: If you add a new file after running this application you'll need to remount the drive in step 1 to make them searchable**

##0. Choose which 'LHI Client' or folder to add transcriptions to

In [35]:
import os

# Reusable function to check and mount Google Drive
def check_and_mount_drive():
    print("Checking /content/drive status...")
    if os.path.exists("/content/drive"):
        print("Mount directory exists. Checking contents...")
        if os.listdir("/content/drive"):
            print("Mountpoint already contains files. Attempting to unmount...")
            try:
                # Unmount the existing mountpoint
                !fusermount -u /content/drive
                print("Unmounted successfully.")
            except Exception as e:
                print(f"Failed to unmount: {e}")
                print("If the issue persists, please select 'Runtime > Disconnect and delete runtime' and try again.")
                return False

    # Mount Google Drive
    print("Mounting Google Drive...")
    from google.colab import drive
    try:
        drive.mount("/content/drive", force_remount=True)
        print("Google Drive mounted successfully.")
    except ValueError as e:
        print(f"Mounting failed: {e}")
        print("If the issue persists, please select 'Runtime > Disconnect and delete runtime' and try again.")
        return False

    # Verify mount
    if os.path.exists("/content/drive/MyDrive"):
        print("Drive is mounted and ready.")
        return True
    else:
        print("Mounting seems incomplete. Please check your drive configuration.")
        return False

# Attempt to check and mount the drive
if check_and_mount_drive():
    print("Proceeding with folder creation...")
else:
    print("Drive mount failed. Cannot proceed.")
    raise SystemExit("Drive mount failed. Exiting.")

# Predefined options for client folders
clients = {
    "1": "/content/drive/MyDrive/Clients/WCBradley/Videos/",
    "2": "/content/drive/MyDrive/Clients/SiriusXM/Videos/",
    "3": "/content/drive/MyDrive/Clients/LHI/Videos/"
}

# Display options to the user
print("Select a client folder:")
print("1: WCBradley")
print("2: SiriusXM")
print("3: LHI")
print("4: Enter a custom folder path")

# Get user input
choice = input("Enter the number corresponding to your choice (default: 1): ").strip()

# Determine the root folder for the client
if choice in clients:
    client_videos_folder = clients[choice]
elif choice == "4":
    client_videos_folder = input("Enter the full path to your Videos folder: ").strip()
else:
    # Default to WCBradley if no valid input
    client_videos_folder = clients["1"]

# Define the WhisperVideo root folder within the client's Videos folder
rootFolder = client_videos_folder + "WhisperVideo/"

# Define subfolder paths relative to the WhisperVideo root folder
audio_folder = rootFolder + "AudioFiles/"
text_folder = rootFolder + "TextFiles/"
processed_folder = rootFolder + "ProcessedVideo/"

# Ensure WhisperVideo folder and its subfolders exist
folders = [rootFolder, audio_folder, text_folder, processed_folder]
for folder in folders:
    try:
        print(f"Checking folder: {folder}")
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f"Created folder: {folder}")
        else:
            print(f"Folder already exists: {folder}")
    except Exception as e:
        print(f"Error ensuring folder {folder}: {e}")

print(f"WhisperVideo folder and subfolders initialized for client:")
print(f"WhisperVideo folder: {rootFolder}")
print(f"Audio files folder: {audio_folder}")
print(f"Text files folder: {text_folder}")
print(f"Processed videos folder: {processed_folder}")

Checking /content/drive status...
Mount directory exists. Checking contents...
Mountpoint already contains files. Attempting to unmount...
Unmounted successfully.
Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully.
Drive is mounted and ready.
Proceeding with folder creation...
Select a client folder:
1: WCBradley
2: SiriusXM
3: LHI
4: Enter a custom folder path
Enter the number corresponding to your choice (default: 1): 
Checking folder: /content/drive/MyDrive/Clients/WCBradley/Videos/WhisperVideo/
Folder already exists: /content/drive/MyDrive/Clients/WCBradley/Videos/WhisperVideo/
Checking folder: /content/drive/MyDrive/Clients/WCBradley/Videos/WhisperVideo/AudioFiles/
Folder already exists: /content/drive/MyDrive/Clients/WCBradley/Videos/WhisperVideo/AudioFiles/
Checking folder: /content/drive/MyDrive/Clients/WCBradley/Videos/WhisperVideo/TextFiles/
Folder already exists: /content/drive/MyDrive/Clients/WCBradley/Videos/WhisperVideo/TextFiles/
Checkin

##1. Load the code libraries

In [36]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa
!pip install audioread

import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
model = whisper.load_model("base.en")
# model = whisper.load_model("small.en") # load the small model
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tui8lk5z
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tui8lk5z
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


##2. Give the application permission to mount the drive and create the folders

In [37]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)  # This will prompt for authorization.

# import os

# # Ensure WhisperVideo folder and its subfolders exist
# folders = [rootFolder, audio_folder, text_folder, processed_folder]
# for folder in folders:
#     try:
#         if not os.path.exists(folder):
#             os.makedirs(folder)
#             print(f"Created folder: {folder}")
#         else:
#             print(f"Folder already exists: {folder}")
#     except Exception as e:
#         print(f"Error ensuring folder {folder}: {e}")

# print(f"All folders verified and ready under: {rootFolder}")

##3. Upload any video files you want transcribed in the "WhisperVideo" folder in your Google Drive.

## 4. Extract audio from the video files and create a transcription

This step processes video files in the `WhisperVideo` folder by extracting audio, transcribing it, and saving the transcription in the `TextFiles` folder. The original video file is moved to the `ProcessedVideo` folder upon successful transcription.

### Shareable Links
The shareable link for the processed video is generated based on its Google Drive file path. This method avoids additional API calls and assumes that files are already shared within your team. The constructed link can be found at the beginning of the transcription file.

Example of a shareable link format:
```
https://drive.google.com/file/d/<file_id>/view
```



In [38]:
import os
import shutil
from datetime import timedelta
import subprocess
import logging
import csv
from datetime import datetime
# Removed: from urllib.parse import quote

# Helper function to format time
def format_time(seconds):
    """Convert seconds to HH:MM:SS format."""
    return str(timedelta(seconds=int(seconds)))

# Removed the generate_shareable_link function

# Setup logging
logging.basicConfig(
    filename="processing_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Ensure folders are created
folders = [rootFolder, audio_folder, text_folder, processed_folder]
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Initialize logs
success_log = []
error_log = []
skipped_log = []

# Process all video files in WhisperVideo folder
video_files = [f for f in os.listdir(rootFolder) if os.path.isfile(os.path.join(rootFolder, f))]

for video_file in video_files:
    # Exclude the report file from processing
    if video_file == "processing_report.txt":
        continue

    # Skip non-video files
    if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
        skipped_log.append((video_file, "Invalid format"))
        print(f"Skipped {video_file}: Invalid format.")
        continue

    # Define paths
    video_path = os.path.join(rootFolder, video_file)
    audio_path = os.path.join(audio_folder, video_file[:-4] + ".wav")
    text_path = os.path.join(text_folder, video_file[:-4] + ".txt")
    processed_path = os.path.join(processed_folder, video_file)

    try:
        print(f"Extracting audio for {video_file} to {audio_path}")
        # Extract audio
        try:
            # Attempt to load the audio using librosa
            y, sr = librosa.load(video_path, sr=16000)  # Load audio with 16 kHz sampling rate
            sf.write(audio_path, y, sr)  # Save audio as a WAV file
            print(f"Audio extraction successful using librosa for {video_file}")
        except Exception as e_librosa:
            print(f"Librosa extraction failed for {video_file}: {e_librosa}")
            print(f"Falling back to ffmpeg for {video_file}")
            # Use ffmpeg as a fallback
            subprocess.run(["ffmpeg", "-i", video_path, "-ar", "16000", "-ac", "1", audio_path], check=True)
            print(f"Audio extraction successful using ffmpeg for {video_file}")

        print(f"Starting transcription for {audio_path}")
        # Transcribe the audio using Whisper
        result = model.transcribe(audio_path)
        print(f"Transcription completed for {audio_path}")

        # Create initial transcription
        text = ""
        for segment in result["segments"]:
            start_time = format_time(segment["start"])
            end_time = format_time(segment["end"])
            text_segment = segment["text"].strip()
            text += f"[{start_time} - {end_time}] {text_segment}\n\n"

        print(f"Saving transcription to {text_path}")
        # Save the transcription
        with open(text_path, "w") as f:
            f.write(text)
        print(f"Transcription saved successfully for {video_file}")

        print(f"Moving file {video_file} to processed folder")
        # Move the video to ProcessedVideo folder
        shutil.move(video_path, processed_path)
        print(f"File moved to processed folder: {processed_path}")

        # Log success
        success_log.append(video_file)
        logging.info(f"Successfully processed {video_file}")
        print(f"Successfully processed {video_file}")

    except subprocess.CalledProcessError as e:
        error_message = f"FFmpeg error for {video_file}: {e}"
        print(error_message)
        error_log.append((video_file, error_message))
        logging.error(error_message)
    except Exception as e:
        error_message = f"General error for {video_file}: {e}"
        print(error_message)
        error_log.append((video_file, error_message))
        logging.error(error_message)

# Perform folder parity check
def get_file_bases(folder):
    return {os.path.splitext(f)[0] for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))}

videos = get_file_bases(processed_folder)
audios = get_file_bases(audio_folder)
texts = get_file_bases(text_folder)

all_match = videos == audios == texts

# Generate completion report
report = "Processing Report\n"
report += f"\nSuccessfully Processed Files ({len(success_log)}):\n"
report += "\n".join(success_log)

report += f"\n\nSkipped Files ({len(skipped_log)}):\n"
report += "\n".join([f"{file} - {reason}" for file, reason in skipped_log])

report += f"\n\nErrors ({len(error_log)}):\n"
report += "\n".join([f"{file} - {reason}" for file, reason in error_log])

report += f"\n\nFolder Parity Check:\n"
report += f"All folders have matching files: {'Yes' if all_match else 'No'}\n"
report += f"Processed Videos: {len(videos)}\n"
report += f"Audio Files: {len(audios)}\n"
report += f"Text Files: {len(texts)}\n"

# Save the report
report_path = os.path.join(rootFolder, "processing_report.txt")
with open(report_path, "w") as f:
    f.write(report)

# Display completion report
print(report)

csv_path = os.path.join(rootFolder, "processing_log.csv")
file_exists = os.path.isfile(csv_path)

# We'll store a timestamp for each run and each file processed/skipped/error
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

with open(csv_path, "a", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    # If file doesn't exist, write header
    if not file_exists:
        writer.writerow(["Timestamp", "FileName", "Status", "Notes"])

    # Append rows for each processed file
    for fname in success_log:
        writer.writerow([current_time, fname, "Processed", ""])

    # Append rows for each skipped file
    for (fname, reason) in skipped_log:
        writer.writerow([current_time, fname, "Skipped", reason])

    # Append rows for each error file
    for (fname, reason) in error_log:
        writer.writerow([current_time, fname, "Error", reason])

print("\nCurrent CSV log entries:")
with open(csv_path, "r", encoding="utf-8") as csvfile:
    print(csvfile.read())

Skipped processing_log.csv: Invalid format.
Processing Report

Successfully Processed Files (0):


Skipped Files (1):
processing_log.csv - Invalid format

Errors (0):


Folder Parity Check:
All folders have matching files: Yes
Processed Videos: 2
Audio Files: 2
Text Files: 2


Current CSV log entries:
Timestamp,FileName,Status,Notes
2024-12-16 03:58:23,Second Standup.mov,Processed,
2024-12-16 03:58:23,Testflight build confusion (v1.5.94 does not contain the code from the commit SHA it references).mov,Processed,
2024-12-16 04:02:22,processing_log.csv,Skipped,Invalid format



In [39]:
# ### Final Note for Synchronization
# For Colab: Sync changes manually after downloading the notebook.
# For Local: Use the Jupytext command:
#    jupytext --sync LHI_WhisperVideoDrive.ipynb

print("Final Note: Synchronize your files locally using Jupytext.")
print("Colab users: Save your notebook and download it to sync manually.")

Jupytext is already installed.
Synchronizing notebook with 'LHI_WhisperVideoDrive.ipynb'...
Error during Jupytext synchronization: Command '['/usr/bin/python3', '-m', 'jupytext', '--sync', 'LHI_WhisperVideoDrive.ipynb']' returned non-zero exit status 1.
